In [4]:
# note: we need to be authenticated via `gcloud auth login`
PROJECT = 'idc-lnq-000'

# CLI access

In [3]:
import os, time

In [3]:
t0 = time.time()
datasets = os.popen('gcloud beta healthcare datasets list --project chc-tcia').read().split()[3::3]
print(time.time() - t0, 'seconds')

2.699720859527588 seconds


In [4]:
t0 = time.time()
for d in datasets[0:1]:
    size = os.popen('gsutil -u '+PROJECT+' du -s gs://gcs-public-data--healthcare-tcia-'+d).read().split()[0]
    print(d, size)
# note: takes a long time
print(time.time() - t0, 'seconds')

4d-lung 183043903814
90.05417990684509 seconds


# Python access

In [1]:
import time
from google.cloud import storage
import google.oauth2.credentials

In [5]:
token = os.popen('gcloud auth print-access-token').read().strip('\n') # TODO: better way?
credentials = google.oauth2.credentials.Credentials(token)

In [6]:
c = storage.Client(project='chc-tcia', credentials=credentials)

In [7]:
t0 = time.time()
datasets = c.list_buckets()
print(time.time() - t0, 'seconds')

0.00016164779663085938 seconds


In [9]:
# getting dataset size
t0 = time.time()
for d in list(datasets)[0:1]:
    bucket = c.bucket(d.name, user_project=PROJECT)
    size = 0
    for b in bucket.list_blobs():
        size += b.size
    print(d, size)
print(time.time() - t0, 'seconds')

<Bucket: gcs-public-data--healthcare-tcia-4d-lung> 183043903814
44.924907207489014 seconds


## Now for the ct-lumph-nodes

In [9]:
search = 'ct-lymph-nodes'

t0 = time.time()
datasets = c.list_buckets()
for d in list(datasets):
    if search in d.name:
        selected_dataset = d.name
        break
print(time.time() - t0, 'seconds')

0.24587154388427734 seconds


In [11]:
bucket = c.bucket(selected_dataset, user_project=PROJECT)

In [12]:
# parsing the folder structure
t0 = time.time()
folders = []
for b in bucket.list_blobs():
    f_split = b.name.split('/')
    for i,f in enumerate(f_split):
        
        if len(folders) <= i:
            folders.append([])
            
        if not f in folders[i]:
            folders[i].append(f)
print(time.time() - t0, 'seconds')

127.93318319320679 seconds


In [22]:
studies = folders[1]
series = folders[2]
images = folders[3]